# Contents
- [Imports](#imports)
- [Data Processing](#process)
- [Modelling](#model)
- [Holdout Testing](#holdout)

---
# Imports<a id=imports></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from inspect import signature
from itertools import cycle
import pickle
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score, make_scorer, roc_curve, classification_report, precision_recall_curve, roc_auc_score

import xgboost as xgb
from xgboost import XGBClassifier

C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework

We load our data that from csvs we had created earlier.

In [2]:
control=pd.read_csv(r'.//data/control_3.807999999999802.csv')
guard=pd.read_csv(r'.//data/Guardclean_7.0959999999994405.csv')
jab=pd.read_csv(r'.//data/Jabclean_1.9500000000000015.csv')
cross=pd.read_csv(r'.//data/Crossclean_2.629999999999932.csv')
hook=pd.read_csv(r'.//data/Hookclean_3.2439999999998643.csv')
upcut=pd.read_csv(r'.//data/Uppercutclean_1.0720000000000007.csv')

---
# Data Processing<a id=process></a>

Let us create our classification labels in our target column called 'move'.

In [3]:
control['move']=0
guard['move']=1
jab['move']=2
cross['move']=3
hook['move']=4
upcut['move']=5

And combine all our different dataframes into one large dataframe.

In [4]:
data=pd.concat([control,guard,jab,cross,hook,upcut],sort=True)
data.head()

,move,x0,x1,x10,x11,x12,x13,x14,x15,x16,...,y16,y17,y2,y3,y4,y5,y6,y7,y8,y9
0,0,0.59,0.53,NaN,0.52,NaN,NaN,0.58,0.59,0.54,...,0.3,NaN,0.42,0.61,0.78,0.42,NaN,NaN,0.75,0.96
1,0,0.59,0.53,NaN,0.55,0.55,NaN,0.58,0.59,0.53,...,0.3,NaN,0.43,0.61,0.75,0.42,NaN,NaN,0.74,0.96
2,0,0.58,0.53,NaN,0.55,0.56,NaN,0.57,0.59,0.53,...,0.3,NaN,0.43,0.61,0.78,0.43,NaN,NaN,0.74,0.96
3,0,0.58,0.52,NaN,0.54,NaN,NaN,0.57,0.58,0.53,...,0.3,NaN,0.43,0.61,0.80,0.43,NaN,NaN,0.73,0.96
4,0,0.58,0.52,NaN,0.53,NaN,NaN,0.57,0.58,0.53,...,0.3,NaN,0.43,0.61,0.78,0.43,NaN,NaN,0.74,0.96


We notice quite a number of NaN values due to the keypoints not being captured in the videos.<br/>
Taking this run of modelling to be the naive case, let us fill all NaN values with 0 which corresponds to the leftmost and topmost coordinates in our video frame.

In [ ]:
#export our dataset for future use
data.to_csv(r'./data/master.csv',index=False)

In [5]:
data.fillna(0,inplace=True)

We separate our dataframe into our X and y sets.

In [6]:
X=data[[x for x in data.columns if x != 'move']]
y=data['move']

And we use SMOTE to upsample our minority classes to ensure an even distribution of our classes.

In [7]:
len(data)

9900

In [8]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

In [9]:
print(str(len(X_res))+' '+str(len(y_res)))

21288 21288


---
# Modelling<a id=model></a>

We perform our train-test-split on our X and y data, stratifying on our classes.

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X_res,y_res,random_state=42,stratify=y_res)

We modify and utilize [code found online](http://www.davidsbatista.net/blog/2018/02/23/model_optimization/) for GridSearching.

In [28]:
#code found from http://www.davidsbatista.net/blog/2018/02/23/model_optimization/
class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=-1, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

We define the classifiers we choose to GridSearch with.

In [32]:
classifier_models = {
    'LogisticRegression' : LogisticRegression(random_state = 42),
    'KNN': KNeighborsClassifier(), 
#     'NaiveBayes' : MultinomialNB(),
    'DecisionTree' : DecisionTreeClassifier(random_state = 42), 
    'BaggedDecisionTree' : BaggingClassifier(random_state = 42),
    'RandomForest' : RandomForestClassifier(random_state = 42), 
    'ExtraTrees' : ExtraTreesClassifier(random_state = 42), 
    'AdaBoost' : AdaBoostClassifier(random_state=42), 
    'GradientBoosting' : GradientBoostingClassifier(random_state = 42),
    'SVM' : SVC(random_state=42),
    'XGBoost' : XGBClassifier(random_state=42)
}

And we set the parameters for us to GridSearch through.

In [33]:
classifier_model_params = {
    'LogisticRegression' : {
        'penalty' : ['l1', 'l2'],
        'C' : np.arange(.05, 1, .05) },
    'KNN' : {
        'n_neighbors' : np.arange(3, 22, 2) },
    'NaiveBayes' : {
        'alpha' : np.arange(.05, 2, .05)},
    'DecisionTree': {
        'max_depth' : [None, 6, 10, 14], 
        'min_samples_leaf' : [1, 2],
        'min_samples_split': [2, 3] },
    'BaggedDecisionTree' : {
        'n_estimators' : [20, 60, 100] },
    'RandomForest' : {
        'n_estimators' : [20, 60, 100],
        'max_depth' : [None, 2, 6, 10],
        'min_samples_split' : [2, 3, 4] },
    'ExtraTrees' : {
        'n_estimators' : [20, 60, 100],
        'max_depth' : [None, 6, 10, 14],
        'min_samples_leaf' : [1, 2], 
        'min_samples_split' : [2, 3], },
    'AdaBoost' : {
        'n_estimators' : np.arange(100, 151, 25),
        'learning_rate' : np.linspace(0.05, 1, 10) },
    'GradientBoosting' : {
        'n_estimators' : np.arange(5, 150, 15),
        'learning_rate' : np.linspace(0.05, 1, 10),
        'max_depth' : [1, 2, 3] },
    'SVM' : {
        'C' : np.arange(0.05, 1, .05),
        'kernel' : ['rbf', 'linear'] },
    'XGBoost' : {
        'n_estimators'  : np.arange(100, 151, 25), 
        'learning_rate' : np.arange(0.1, 1, .3),
        'max_depth' : [3],
        'alpha' : np.arange(0, 1, .3),
        'lambda' : np.arange(0, 1, .3),
        'gamma' : np.arange(0, 1, .3),
        'subsample' : [.5],
        'n_jobs' : [4],}
        }

In [34]:
search = EstimatorSelectionHelper(classifier_models, classifier_model_params)
search.fit(X_train, y_train, scoring='accuracy', n_jobs=-1)

Running GridSearchCV for LogisticRegression.
Fitting 3 folds for each of 38 candidates, totalling 114 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done 114 out of 114 | elapsed:  2.8min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for KNN.
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   38.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for DecisionTree.
Fitting 3 folds for each of 16 candidates, totalling 48 fits


...

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   44.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 23.8min
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed: 26.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for SVM.
Fitting 3 folds for each of 38 candidates, totalling 114 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 114 out of 114 | elapsed:  3.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for XGBoost.
Fitting 3 folds for each of 576 candidates, totalling 1728 fits


C:\Users\chang\.conda\envs\opencv\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 33.6min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 53.1min
[Parallel(n_jobs=-1)]: Done 1728 out of 1728 | elapsed: 73.7min finished


---
# Holdout Testing<a id=holdout></a>
We create a table of our GridSearch scores and parameters.

In [35]:
score1=search.score_summary(sort_by='mean_score')

LogisticRegression
KNN
DecisionTree
BaggedDecisionTree
RandomForest
ExtraTrees
AdaBoost
GradientBoosting
SVM
XGBoost


C:\Users\chang\.conda\envs\opencv\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



And we create a pivot table which gives us the best scores and the parameters for each of our estimators.

In [36]:
table=pd.pivot_table(score1,index='estimator',aggfunc='max').sort_values('mean_score',ascending=False)
table

,C,alpha,gamma,kernel,lambda,learning_rate,max_depth,max_score,mean_score,min_samples_leaf,min_samples_split,min_score,n_estimators,n_jobs,n_neighbors,penalty,std_score,subsample
estimator,,,,,,,,,,,,,,,,,,
XGBoost,NaN,0.9,0.9,NaN,0.9,0.7,3.0,0.998873,0.998246,NaN,NaN,0.997745,150.0,4.0,NaN,NaN,0.000998,0.5
ExtraTrees,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.998873,0.998058,2.0,3.0,0.997557,100.0,NaN,NaN,NaN,0.006249,NaN
GradientBoosting,NaN,NaN,NaN,NaN,NaN,1.0,3.0,0.998685,0.997933,NaN,NaN,0.997557,140.0,NaN,NaN,NaN,0.008718,NaN
RandomForest,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.998497,0.997369,NaN,4.0,0.996618,100.0,NaN,NaN,NaN,0.007101,NaN
KNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.994175,0.993048,NaN,NaN,0.992296,NaN,NaN,21.0,NaN,0.002002,NaN
BaggedDecisionTree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.993611,0.992296,NaN,NaN,0.990605,100.0,NaN,NaN,NaN,0.001602,NaN
DecisionTree,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.988350,0.987974,2.0,3.0,0.987787,NaN,NaN,NaN,NaN,0.003418,NaN
LogisticRegression,0.95,NaN,NaN,NaN,NaN,NaN,NaN,0.958286,0.956658,NaN,NaN,0.954904,NaN,NaN,NaN,l2,0.003386,NaN
SVM,0.95,NaN,NaN,rbf,NaN,NaN,NaN,0.956219,0.954340,NaN,NaN,0.950582,NaN,NaN,NaN,NaN,0.009283,NaN


We export these parameters to an external csv file for future reference if needed.

In [37]:
table.to_csv(r'.\data\model1params.csv')

In [38]:
table=pd.read_csv(r'.\data\model1params.csv',index_col='estimator')
table

,C,alpha,gamma,kernel,lambda,learning_rate,max_depth,max_score,mean_score,min_samples_leaf,min_samples_split,min_score,n_estimators,n_jobs,n_neighbors,penalty,std_score,subsample
estimator,,,,,,,,,,,,,,,,,,
XGBoost,NaN,0.9,0.9,NaN,0.9,0.7,3.0,0.998873,0.998246,NaN,NaN,0.997745,150.0,4.0,NaN,NaN,0.000998,0.5
ExtraTrees,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.998873,0.998058,2.0,3.0,0.997557,100.0,NaN,NaN,NaN,0.006249,NaN
GradientBoosting,NaN,NaN,NaN,NaN,NaN,1.0,3.0,0.998685,0.997933,NaN,NaN,0.997557,140.0,NaN,NaN,NaN,0.008718,NaN
RandomForest,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.998497,0.997369,NaN,4.0,0.996618,100.0,NaN,NaN,NaN,0.007101,NaN
KNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.994175,0.993048,NaN,NaN,0.992296,NaN,NaN,21.0,NaN,0.002002,NaN
BaggedDecisionTree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.993611,0.992296,NaN,NaN,0.990605,100.0,NaN,NaN,NaN,0.001602,NaN
DecisionTree,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.988350,0.987974,2.0,3.0,0.987787,NaN,NaN,NaN,NaN,0.003418,NaN
LogisticRegression,0.95,NaN,NaN,NaN,NaN,NaN,NaN,0.958286,0.956658,NaN,NaN,0.954904,NaN,NaN,NaN,l2,0.003386,NaN
SVM,0.95,NaN,NaN,rbf,NaN,NaN,NaN,0.956219,0.954340,NaN,NaN,0.950582,NaN,NaN,NaN,NaN,0.009283,NaN


We define the estimators we will use for our holdout testing based on the parameters that gave us the best accuracy stores in our GridSearch.

In [41]:
estimators = {
    'XGB' : XGBClassifier(reg_alpha=table.loc['XGBoost']['alpha'],reg_lambda=table.loc['XGBoost']['lambda'],random_state = 42, gamma=table.loc['XGBoost']['gamma'],learning_rate=table.loc['XGBoost']['learning_rate'],max_depth=int(table.loc['XGBoost']['max_depth']),n_estimators=int(table.loc['XGBoost']['n_estimators']),subsample=table.loc['XGBoost']['subsample']),
    'ADA' : AdaBoostClassifier(random_state = 42,learning_rate=table.loc['AdaBoost']['learning_rate'],n_estimators=int(table.loc['AdaBoost']['n_estimators'])),
    'GBoost' : GradientBoostingClassifier(random_state = 42,learning_rate=table.loc['GradientBoosting']['learning_rate'],max_depth=int(table.loc['GradientBoosting']['max_depth']),n_estimators=int(table.loc['GradientBoosting']['n_estimators'])),
    'SVC' : SVC(C=table.loc['SVM']['C'],random_state = 42,kernel=table.loc['SVM']['kernel'],probability=True,gamma='auto'),
    'DecisionTree' : DecisionTreeClassifier(random_state = 42,max_depth=int(table.loc['DecisionTree']['max_depth']),min_samples_leaf=int(table.loc['DecisionTree']['min_samples_leaf']),min_samples_split=int(table.loc['DecisionTree']['min_samples_split'])),
    'knn' : KNeighborsClassifier(n_neighbors=int(table.loc['KNN']['n_neighbors'])),
    'random forest' : RandomForestClassifier(random_state = 42,max_depth=int(table.loc['RandomForest']['max_depth']),min_samples_split=int(table.loc['RandomForest']['min_samples_split']),n_estimators=int(table.loc['RandomForest']['n_estimators'])),
    'ef' : ExtraTreesClassifier(random_state = 42,max_depth=int(table.loc['ExtraTrees']['max_depth']),min_samples_leaf=int(table.loc['ExtraTrees']['min_samples_leaf']),min_samples_split=int(table.loc['ExtraTrees']['min_samples_split']),n_estimators=int(table.loc['ExtraTrees']['n_estimators'])),
    'lr with reg' : LogisticRegression(random_state = 42,penalty=table.loc['LogisticRegression']['penalty'],C=table.loc['LogisticRegression']['C']),
    'bagging classifier' : BaggingClassifier(random_state = 42,n_estimators=int(table.loc['BaggedDecisionTree']['n_estimators'])),
#     'NaiveBayes' : MultinomialNB(alpha=table.loc['NaiveBayes']['alpha'])
}

And we perform our holdout testing for the previously defined estimators.

In [43]:
scores = []

for e_name,estimator in estimators.items():
    estimator.fit(X_train,y_train)
    pred=estimator.predict(X_test)
    fpr, tpr, _ = roc_curve(y_test, estimator.predict_proba(X_test)[:,1],pos_label=1)
    precision, recall, _ = precision_recall_curve(y_test, estimator.predict_proba(X_test)[:,1],pos_label=1)
    score = {'est':e_name,
             'roc_curve':[fpr, tpr],
             'prc':[precision,recall]}
    scores.append(score)
    print('Results for ',e_name,':')
    print(classification_report(y_test,pred))


Results for  XGB :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       887
           1       1.00      1.00      1.00       887
           2       1.00      1.00      1.00       887
           3       1.00      1.00      1.00       887
           4       1.00      0.99      1.00       887
           5       0.99      1.00      1.00       887

    accuracy                           1.00      5322
   macro avg       1.00      1.00      1.00      5322
weighted avg       1.00      1.00      1.00      5322

Results for  ADA :
              precision    recall  f1-score   support

           0       0.47      0.71      0.57       887
           1       0.47      0.98      0.64       887
           2       0.71      0.08      0.15       887
           3       0.57      0.97      0.71       887
           4       0.44      0.07      0.12       887
           5       0.78      0.34      0.47       887

    accuracy                           

C:\Users\chang\.conda\envs\opencv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chang\.conda\envs\opencv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Results for  lr with reg :
              precision    recall  f1-score   support

           0       0.95      0.86      0.91       887
           1       0.90      0.97      0.94       887
           2       0.95      0.95      0.95       887
           3       0.97      0.99      0.98       887
           4       0.94      0.89      0.91       887
           5       0.88      0.93      0.90       887

    accuracy                           0.93      5322
   macro avg       0.93      0.93      0.93      5322
weighted avg       0.93      0.93      0.93      5322

Results for  bagging classifier :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       887
           1       0.99      1.00      1.00       887
           2       1.00      1.00      1.00       887
           3       1.00      1.00      1.00       887
           4       1.00      0.99      0.99       887
           5       0.99      0.99      0.99       887

    accuracy    

Observing how our models perform on our holdout sets, it seems that some models are performing admirably.<br/>
XGBoost,Gradient Boosting, Random Forest, Extra Trees and Bagging Classifier models give us close to perfect accuracy, but it brings up the issue of overfitting which we will have to test later on.<br/>
The other models also performed well with the exception of the AdaBoost model. This might possibly be a result of restricted parameters in our GridSearch.<br/><br/>
Now that we have fitted all our best models, we shall pickle the models for testing purposes later on.

In [44]:
for e_name, estimator in estimators.items():
    pickle.dump(estimator,open('./data/model1/'+str(e_name)+'.sav', 'wb'))

And we test if our pickled files are working correctly.

In [45]:
model='XGB'

In [46]:
mod=pickle.load(open(r'.//data/model1/'+model+'.sav', 'rb'))

In [57]:
result=mod.predict(X_test)

In [63]:
compare=estimators['XGB'].predict(X_test)

In [67]:
for i in range(len(result)):
    if result[i]==compare[i]:
        continue
    else:
        print('mismatch')
print('done')

done


It seems that our pickling was successful. We shall move on to testing our models.